In [15]:
pip install gradio langchain transformers torch accelerate


In [16]:
pip install langchain_community

In [17]:
pip install autogen

In [18]:
pip install groq

In [ ]:
# check
# ---- 1️⃣  Install dependencies ----
!pip install gradio python-dotenv requests

# ---- 2️⃣  Enter your API keys here (keep them private!) ----
import os
os.environ["GROQ_API_KEY"]   = ""
os.environ["TAVILY_API_KEY"] = ""
os.environ["SERPER_API_KEY"]  = ""
os.environ["OPENWEATHER_KEY"] = ""
# ---- 3️⃣  Travel-planner agents ----
import json, requests, gradio as gr
from datetime import datetime, timedelta

In [20]:
# tavily not used

GROQ_KEY   = os.getenv("GROQ_API_KEY")
TAVILY_KEY= os.getenv("TAVILY_API_KEY")
SERPER_KEY = os.getenv("SERPER_API_KEY")
OPENWEATHER_KEY = os.getenv("OPENWEATHER_KEY")

In [22]:
import gradio as gr, requests, json

# ---------- Formatters ----------
def fmt_list(items, icon="•"):
    """Format list of dicts with title + snippet nicely."""
    if not items: return f"{icon} No data found."
    return "\n\n".join(
        f"{icon} **{it.get('title','No title')}**\n   {it.get('snippet','')}"
        for it in items
    )

# ---------- Agents ----------
class FlightAgent:
    def search(self, origin, destination, date):
        q = f"{origin} to {destination} flights {date}"
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": SERPER_KEY, "Content-Type": "application/json"}
        r = requests.post(url, headers=headers, json={"q": q}, timeout=20).json()
        return r.get("organic", [])[:5]

class HotelAgent:
    def search(self, city, checkin, checkout):
        q = f"hotels in {city} check-in {checkin} check-out {checkout}"
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": SERPER_KEY, "Content-Type": "application/json"}
        r = requests.post(url, headers=headers, json={"q": q}, timeout=20).json()
        return r.get("organic", [])[:5]

class SummaryAgent:
    def summarize(self, origin, destination, flights, hotels, notes):
        prompt = (
            f"Create a friendly short trip plan from {origin} to {destination}.\n"
            f"Flights:\n{json.dumps(flights, indent=2)}\n\n"
            f"Hotels:\n{json.dumps(hotels, indent=2)}\n\n"
            f"Traveler notes: {notes}"
        )
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {"Authorization": f"Bearer {GROQ_KEY}"}
        payload = {"model": "llama-3.1-8b-instant",
                   "messages": [{"role": "user", "content": prompt}],
                   "temperature": 0.7}
        r = requests.post(url, headers=headers, json=payload, timeout=60).json()
        if "choices" in r and r["choices"]:
            return r["choices"][0]["message"]["content"]
        return "⚠️ Unable to generate summary."

class WeatherAgent:
    def get_weather(self, city):
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={OPENWEATHER_KEY}"
        try:
            r = requests.get(url, timeout=10).json()
            if r.get("cod") != 200:
                return f"🌦 Weather data not available for {city}"
            desc = r["weather"][0]["description"].capitalize()
            temp = r["main"]["temp"]
            feels = r["main"]["feels_like"]
            hum = r["main"]["humidity"]
            wind = r["wind"]["speed"]
            return (
                f"🌦 **Weather in {city}:**\n"
                f"• {desc}\n"
                f"• Temperature: {temp}°C (feels like {feels}°C)\n"
                f"• Humidity: {hum}%\n"
                f"• Wind speed: {wind} m/s"
            )
        except Exception as e:
            return f"🌦 Error fetching weather: {e}"

# ---------- Planner ----------
class PlannerAgent:
    def __init__(self):
        self.flight  = FlightAgent()
        self.hotel   = HotelAgent()
        self.summary = SummaryAgent()
        self.weather = WeatherAgent()

    def plan(self, origin, destination, depart, ret, notes, include_weather=False):
        flights = self.flight.search(origin, destination, depart)
        hotels  = self.hotel.search(destination, depart, ret)
        summary = self.summary.summarize(origin, destination, flights, hotels, notes)
        weather_info = self.weather.get_weather(destination) if include_weather else ""
        return summary, flights, hotels, weather_info

planner = PlannerAgent()

# ---------- Runners ----------
def run_all(origin, destination, depart_date, return_date, notes, weather_toggle):
    include_weather = weather_toggle == "Yes"
    summary, flights, hotels, weather_info = planner.plan(
        origin, destination, depart_date, return_date, notes, include_weather
    )
    return (
        f"## 🗺 Trip Summary\n{summary}",
        f"### 🛫 Flights\n{fmt_list(flights)}",
        f"### 🏨 Hotels\n{fmt_list(hotels)}",
        weather_info if include_weather else "🌦 Weather not requested."
    )

def run_single(agent, origin, destination, depart_date, return_date, notes):
    if agent == "Flight":
        return f"### 🛫 Flights\n{fmt_list(planner.flight.search(origin, destination, depart_date))}"
    elif agent == "Hotel":
        return f"### 🏨 Hotels\n{fmt_list(planner.hotel.search(destination, depart_date, return_date))}"
    elif agent == "Summary":
        flights = planner.flight.search(origin, destination, depart_date)
        hotels  = planner.hotel.search(destination, depart_date, return_date)
        return f"## 🗺 Trip Summary\n{planner.summary.summarize(origin, destination, flights, hotels, notes)}"
    elif agent == "Weather":
        return planner.weather.get_weather(destination)
    else:
        return "Please choose a valid agent."

# ---------- UI ----------
with gr.Blocks(theme="soft", title="✈️ Travel Planner Agents") as demo:
    gr.Markdown("# ✈️ **Smart Travel Planner**")
    gr.Markdown("Plan your trip with multi-agent intelligence: Flights, Hotels, Summary and optional Weather.")

    with gr.Tabs():
        # ----- Multi-Agent -----
        with gr.TabItem("🧭 Multi-Agent Planner"):
            with gr.Row():
                with gr.Column(scale=2):
                    origin1 = gr.Textbox(label="🌏 Origin")
                    dest1   = gr.Textbox(label="📍 Destination")
                    depart1 = gr.Textbox(label="📅 Departure (YYYY-MM-DD)")
                    ret1    = gr.Textbox(label="📅 Return (YYYY-MM-DD)")
                    notes1  = gr.Textbox(label="📝 Notes / Preferences")
                    #weather_toggle = gr.Checkbox(label="🌦 Include Weather Info", value=False)
                    weather_toggle = gr.Dropdown(["No", "Yes"], value="No",
                                                label="🌦 Include Weather Info?")
                    btn1    = gr.Button("🚀 Plan Trip", variant="primary")
                with gr.Column(scale=3):
                    out_summary1 = gr.Markdown()
                    out_flights1 = gr.Markdown()
                    out_hotels1  = gr.Markdown()
                    out_weather1 = gr.Markdown()
            btn1.click(run_all,
                       inputs=[origin1, dest1, depart1, ret1, notes1, weather_toggle],
                       outputs=[out_summary1, out_flights1, out_hotels1, out_weather1])

        # ----- Single Agent -----
        with gr.TabItem("🎯 Single Agent Runner"):
            gr.Markdown("Run a **single agent** independently.")
            agent_choice = gr.Radio(["Flight", "Hotel", "Summary", "Weather"],
                                    label="Select Agent")
            with gr.Row():
                with gr.Column(scale=2):
                    origin2 = gr.Textbox(label="🌏 Origin")
                    dest2   = gr.Textbox(label="📍 Destination")
                    depart2 = gr.Textbox(label="📅 Departure (YYYY-MM-DD)")
                    ret2    = gr.Textbox(label="📅 Return (YYYY-MM-DD)")
                    notes2  = gr.Textbox(label="📝 Notes / Preferences")
                    btn2    = gr.Button("▶ Run Selected Agent", variant="primary")
                with gr.Column(scale=3):
                    out_single = gr.Markdown()
            btn2.click(run_single,
                       inputs=[agent_choice, origin2, dest2, depart2, ret2, notes2],
                       outputs=[out_single])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a59f9b0ee001fae8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
